# Dlaczego wasz model jest lepszy od innych? 


Otrzymaliśmy 92% trafności na danych testowych. Uważamy, że model jest wystarczająco dobry, żeby móc go zaimplementować

# Jak to sprawdziliście? 


Model przetrenowaliśmy na oczyszczonych danych (9826 pomiarów). Zawsze otrzymaliśmy skuteczność około 91%

# Dlaczego podjęliście takie a nie inne decyzje?


Chcieliśmy uprościć problem jak najbardziej. Dlatego podczas oczyszczania danych usunęliśmy wszystkie wiersze, które zawierały NaNy - błędne pomiary nie powinny być interpretowane przez model. Następnie wybraliśmy tylko wartości określone w kolumnach jako średnia (mean) lub odchylenie standardowe (std) - czyli najważniejsze dla nas wartości. Dzięki temu zmnieszyliśmy wymiarowość danych do 66 cech. Żeby sieć nie uczyła się też na danych, które zostały błędnie odczytane, obliczyliśmy dla każdej osoby (subject) w ramach każdej aktywności, wartości średnie dla wspomnianych wcześniej cech. Dało to nam 160 pomiarów treningowych. To wystarczyło sieci neuronowej na bardzo efektywną naukę.

Użyliśmy funkcji aktywacji softmax na końcu ze względu na jej charakter. Zwraca nam ona listę prawdopodobieństw, która sumuje się do 1. Można dzięki temu interpretować, które aktywności są najbardziej podobne do danego pomiaru, a które mniej

Kryteria oceny:
1. Czyszczenie danych i redukcja wymiarowości. (10 pkt)*
2. Dokładność modelu. (25 pkt)
3. Zrozumienie i interpretacja wyników. (35 pkt)*
4. Prezentacja i użyteczność biznesowa. (20 pkt)

## Wstępna analiza

Wczytujemy plik csv używając biblioteki pandas jako DataFrame, upewniamy się że widoczne będą wszystkie kolumny z danymi, analizujemy wielkość i naturę zbioru danych.

In [1]:
import pandas as pd
df = pd.read_csv('data/final_train.csv')
pd.set_option('display.max_columns', None)

In [2]:
df.shape

(11086, 573)

In [3]:
df.sample(6)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16","fBodyAcc-bandsEnergy()-1,16.1","fBodyAcc-bandsEnergy()-1,16.2","fBodyAcc-bandsEnergy()-1,24","fBodyAcc-bandsEnergy()-1,24.1","fBodyAcc-bandsEnergy()-1,24.2","fBodyAcc-bandsEnergy()-1,8","fBodyAcc-bandsEnergy()-1,8.1","fBodyAcc-bandsEnergy()-1,8.2","fBodyAcc-bandsEnergy()-17,24","fBodyAcc-bandsEnergy()-17,24.1","fBodyAcc-bandsEnergy()-17,24.2","fBodyAcc-bandsEnergy()-17,32","fBodyAcc-bandsEnergy()-17,32.1","fBodyAcc-bandsEnergy()-17,32.2","fBodyAcc-bandsEnergy()-25,32","fBodyAcc-bandsEnergy()-25,32.1","fBodyAcc-bandsEnergy()-25,32.2","fBodyAcc-bandsEnergy()-25,48","fBodyAcc-bandsEnergy()-25,48.1","fBodyAcc-bandsEnergy()-25,48.2","fBodyAcc-bandsEnergy()-33,40","fBodyAcc-bandsEnergy()-33,40.1","fBodyAcc-bandsEnergy()-33,40.2","fBodyAcc-bandsEnergy()-33,48","fBodyAcc-bandsEnergy()-33,48.1","fBodyAcc-bandsEnergy()-33,48.2","fBodyAcc-bandsEnergy()-41,48","fBodyAcc-bandsEnergy()-41,48.1","fBodyAcc-bandsEnergy()-41,48.2","fBodyAcc-bandsEnergy()-49,56","fBodyAcc-bandsEnergy()-49,56.1","fBodyAcc-bandsEnergy()-49,56.2","fBodyAcc-bandsEnergy()-49,64","fBodyAcc-bandsEnergy()-49,64.1","fBodyAcc-bandsEnergy()-49,64.2","fBodyAcc-bandsEnergy()-57,64","fBodyAcc-bandsEnergy()-57,64.1","fBodyAcc-bandsEnergy()-57,64.2","fBodyAcc-bandsEnergy()-9,16","fBodyAcc-bandsEnergy()-9,16.1","fBodyAcc-bandsEnergy()-9,16.2",fBodyAcc-energy()-X,fBodyAcc-energy()-Y,fBodyAcc-energy()-Z,fBodyAcc-entropy()-X,fBodyAcc-entropy()-Y,fBodyAcc-entropy()-Z,fBodyAcc-iqr()-X,fBodyAcc-iqr()-Y,fBodyAcc-iqr()-Z,fBodyAcc-kurtosis()-X,fBodyAcc-kurtosis()-Y,fBodyAcc-kurtosis()-Z,fBodyAcc-mad()-X,fBodyAcc-mad()-Y,fBodyAcc-mad()-Z,fBodyAcc-main()-X,fBodyAcc-main()-Y,fBodyAcc-main()-Z,fBodyAcc-max()-X,fBodyAcc-max()-Y,fBodyAcc-max()-Z,fBodyAcc-maxInds-X,fBodyAcc-maxInds-Y,fBodyAcc-maxInds-Z,fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAcc-min()-X,fBodyAcc-min()-Y,fBodyAcc-min()-Z,fBodyAcc-skewness()-X,fBodyAcc-skewness()-Y,fBodyAcc-skewness()-Z,fBodyAcc-sma(),fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,"fBodyAccJerk-bandsEnergy()-1,16","fBodyAccJerk-bandsEnergy()-1,16.1","fBodyAccJerk-bandsEnergy()-1,16.2","fBodyAccJerk-bandsEnergy()-1,24","fBodyAccJerk-bandsEnergy()-1,24.1","fBodyAccJerk-bandsEnergy()-1,24.2","fBodyAccJerk-bandsEnergy()-1,8","fBodyAccJerk-bandsEnergy()-1,8.1","fBodyAccJerk-bandsEnergy()-1,8.2","fBodyAccJerk-bandsEnergy()-17,24","fBodyAccJerk-bandsEnergy()-17,24.1","fBodyAccJerk-bandsEnergy()-17,24.2","fBodyAccJerk-bandsEnergy()-17,32","fBodyAccJerk-bandsEnergy()-17,32.1","fBodyAccJerk-bandsEnergy()-17,32.2","fBodyAccJerk-bandsEnergy()-25,32","fBodyAccJerk-bandsEnergy()-25,32.1","fBodyAccJerk-bandsEnergy()-25,32.2","fBodyAccJerk-bandsEnergy()-25,48","fBodyAccJerk-bandsEnergy()-25,48.1","fBodyAccJerk-bandsEnergy()-25,48.2","fBodyAccJerk-bandsEnergy()-33,40","fBodyAccJerk-bandsEnergy()-33,40.1","fBodyAccJerk-bandsEnergy()-33,40.2","fBodyAccJerk-bandsEnergy()-33,48","fBodyAccJerk-bandsEnergy()-33,48.1","fBodyAccJerk-bandsEnergy()-33,48.2","fBodyAccJerk-bandsEnergy()-41,48","fBodyAccJerk-bandsEnergy()-41,48.1","fBodyAccJerk-bandsEnergy()-41,48.2","fBodyAccJerk-bandsEnergy()-49,56","fBodyAccJerk-bandsEnergy()-49,56.1","fBodyAccJerk-bandsEnergy()-49,56.2","fBodyAccJerk-bandsEnergy()-49,64","fBodyAccJerk-bandsEnergy()-49,64.1","fBodyAccJerk-bandsEnergy()-49,64.2","fBodyAccJerk-bandsEnergy()-57,64","fBodyAccJerk-bandsEnergy()-57,64.1","fBodyAccJerk-bandsEnergy()-57,64.2","fBodyAccJerk-bandsEnergy()-9,16","fBodyAccJerk-bandsEnergy()-9,16.1","fBodyAccJerk-bandsEnergy()-9,16.2",fBodyAccJerk-energy()-X,fBodyAccJerk-energy()-Y,fBodyAccJerk-energy()-Z,fBodyAccJerk-entropy()-X,fBodyAccJerk-entropy()-Y,fBodyAccJerk-entropy(

W trakcie analizy dochodzimy do kilku wniosków, mianowicie:

In [4]:
pd.unique(df['Activity'])

array(['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING',
       'WALKING_DOWNSTAIRS'], dtype=object)

1. Mamy 6 aktywności, które nasz model ma rozpoznawać,

In [5]:
df.subject.nunique()

30

2. W badaniu brało udział 30 uczestników,

3. Do trenowania modelu klasyfikacji definitywnie nie potrzebujemy wszystkich cech. Skupiamy się na tych, które zawierają średnią i odchylenie standardowe.

## Oczyszczanie danych

Zanim zaczniemy czyścić zbiór, wyciągamy z niego dane o uczestnikach i aktywnościach, to przyda nam się potem.

In [6]:
subject = df['subject']
activity = df['Activity']

Na początku wybieramy wszystkie kolumny zawierające średnią lub odchylenie standardowe wszystkich z cech.

In [7]:
mean_cols = [col for col in df.columns if 'mean' in col or 'Mean' in col]
std_cols = [col for col in df.columns if 'std' in col or 'Std' in col]

df = pd.concat([df[std_cols], df[mean_cols]], axis = 1)
df.shape

(11086, 87)

In [8]:
573-87, (573-87)/573

(486, 0.8481675392670157)

W ten sposób udało nam się pozbyć 486 cech, co stanowi prawie 85% całego zbioru.

Aktualnie nasz zbiór wygląda tak:

In [9]:
df.sample(4)

,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccJerk-meanFreq()-X,fBodyAccJerk-meanFreq()-Y,fBodyAccJerk-meanFreq()-Z,fBodyAccMag-mean(),fBodyAccMag-meanFreq(),fBodyBodyAccJerkMag-mean(),fBodyBodyAccJerkMag-meanFreq(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-meanFreq(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,fBodyGyro-meanFreq()-X,fBodyGyro-meanFreq()-Y,fBodyGyro-meanFreq()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
5831,0.797941,-1.267550,4.182218,-4.028178,0.261975,2.359286,-0.816908,-3.016575,-6.016181,4.832826,3.221559,0.804087,4.685762,6.346211,3.884976,3.394077,6.312624,-0.522479,3.924207,2.310968,1.625686,5.283754,5.552676,1.774669,-4.156204,-0.589930,0.852433,-2.665807,5.710371,3.742629,1.438680,4.079544,2.026792,-1.574933,-0.606069,-2.462062,-1.862981,0.305753,0.838841,6.729408,-6.362769,-7.614423,-1.032901,6.363068,-3.925607,3.177884,-1.907578,4.687129,1.787779,-1.614689,-4.371580,6.798716,1.209301,7.512510,3.758461,3.336348,0.741059,-0.317665,0.297146,1.600659,3.205798,1.466039,2.531925,-5.505099,0.167475,0.453852,0.585594,3.987793,-0.953676,2.877671,0.771910,3.544043,1.753072,5.769300,3.571589,-2.084927,3.181885,-2.197997,1.118894,0.476480,2.307929,2.681118,0.305760,2.250625,2.375269,4.325139,-0.940920
5345,-1.223073,-3.158632,3.912448,-5.525108,2.343067,1.473819,-2.797628,-2.847767,-6.075579,4.619765,1.194675,0.740399,4.213907,5.690689,3.738668,2.671849,5.187333,-2.401673,3.330657,3.083971,-1.287540,3.295311,3.772891,1.242333,-3.156759,-0.494562,0.854299,-2.337315,3.988794,3.693668,1.272762,4.128248,-1.447174,-1.573969,-0.712497,-2.444000,-1.913643,0.317868,-1.429880,1.903187,-4.042786,-7.251313,-0.402270,6.980440,-2.741586,3.927016,-1.939520,3.389079,-0.482508,-2.471304,-5.830207,7.213756,2.065014,5.651956,4.074944,3.320713,1.985455,-0.729945,0.504128,1.972142,2.638049,2.335678,2.488841,-3.726814,0.691710,2.006007,0.881108,3.940340,-1.029721,2.894356,0.694499,3.510903,1.931073,4.332225,4.520839,-1.509082,3.076884,-1.975462,1.084552,0.329481,3.319433,2.049329,-1.181769,2.272891,2.633657,3.212502,-0.798089
2428,0.799663,-1.348191,4.212089,-4.034683,0.277929,2.364314,-0.832648,-3.015946,-6.016224,4.839664,3.250277,0.812255,4.709953,6.346384,3.879430,3.455001,6.307238,-0.539922,3.926187,2.314490,1.612246,5.309700,5.719632,1.795215,-4.162492,-0.590106,0.852426,-2.666165,5.760531,3.721022,1.347543,4.101103,2.010764,-1.572155,-0.835082,-2.469389,-1.834504,0.064988,-0.168543,8.055053,-12.025193,-7.614058,-1.014427,6.336656,-4.607460,4.232298,-1.920507,4.680374,1.76

Dodatkowo zdecydowaliśmy się pozbyć kolumn, które zawierają słowa kluczowe: 'angle', 'energy' i 'meanFreq'

In [10]:
angle_cols = [col for col in df.columns if 'angle' in col or 'Angle' in col]
energy_cols = [col for col in df.columns if 'energy' in col or 'Energy' in col]
meanFreq_cols = [col for col in df.columns if 'meanFreq' in col]

drop_cols = angle_cols + energy_cols + meanFreq_cols
df = df.drop(drop_cols, axis=1)

In [11]:
df.shape

(11086, 66)

Ostatecznie nasz zbiór ma 66 cech.

Dodajemy kolumny uczestnika i aktywności.

In [12]:
df.insert(0, 'Activity', activity)
df.insert(0, 'Subject', subject)

Pozbywamy się niekompletnych wierszy.

In [ ]:
df = df.dropna()

Ostatecznie zdecydowaliśmy się zgrupować wszystkich uczestników badania razem z aktywnościami, w których brali udział

In [ ]:
grouped = df.groupby(['Subject', 'Activity'])

Wyciągamy średnią z wszystkich cech, dla każdego uczestnika względem każdej z badanych aktywności.

In [ ]:
df = grouped.mean().reset_index()

I tak prezentują się nasze oczyszczone dane:

In [ ]:
df.head(10)

In [ ]:
d